In [1]:
import cv2
import keras 
import os
import numpy as np
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Convolution2D 
from keras.layers import Dense
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

Using TensorFlow backend.
C:\Users\kushagra\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kushagra\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kushagra\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kushagra\Anaconda3\lib\site-packages\tensorflow\python\framewo

''' Building a CNN model with 2 layers and number of channels taken in convolutional layer(first) are 3. Also we are taking the shape of input image as 100 * 100. '''

In [2]:
classifier = Sequential()

In [3]:
classifier.add(Convolution2D(100,3,3,input_shape = (100,100,3),activation = 'relu'))

In [4]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [5]:
classifier.add(Convolution2D(100,3,3,activation = 'relu'))

In [6]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [7]:
classifier.add(Flatten())

In [8]:
classifier.add(Dropout(0.5))

In [9]:
classifier.add(Dense(output_dim = 128,activation = 'relu'))

In [10]:
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

In [11]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


'''Doing data augmentation to get more images to train our model but also converting each augmented image to size of 100 * 100.'''

In [12]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('train',target_size = (100,100),batch_size = 32,class_mode = 'binary')
test_set = test_datagen.flow_from_directory('test',target_size = (100,100),batch_size = 32,class_mode = 'binary')

classifier.fit_generator(training_set,samples_per_epoch = 87,nb_epoch = 25,validation_data = test_set,nb_val_samples = 13)

Found 174 images belonging to 2 classes.
Found 26 images belonging to 2 classes.

Epoch 1/25
2/2 [==============================] - 11s 6s/step - loss: 1.2614 - accuracy: 0.5781 - val_loss: 2.7872 - val_accuracy: 0.5000
Epoch 2/25
2/2 [==============================] - 9s 5s/step - loss: 2.2443 - accuracy: 0.4348 - val_loss: 0.7482 - val_accuracy: 0.5000
Epoch 3/25
2/2 [==============================] - 11s 6s/step - loss: 0.7268 - accuracy: 0.5781 - val_loss: 0.6911 - val_accuracy: 0.5000
Epoch 4/25
2/2 [==============================] - 10s 5s/step - loss: 0.6884 - accuracy: 0.6957 - val_loss: 0.6811 - val_accuracy: 0.6923
Epoch 5/25
2/2 [==============================] - 11s 6s/step - loss: 0.6802 - accuracy: 0.6094 - val_loss: 0.6557 - val_accuracy: 0.5000
Epoch 6/25
2/2 [==============================] - 12s 6s/step - loss: 0.6627 - accuracy: 0.5312 - val_loss: 0.6158 - val_accuracy: 0.9231
Epoch 7/25
2/2 [==============================] - 12s 6s/step - loss: 0.5934 - accuracy: 0.

In [13]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
label_dict = {0:'without mask',1:'with mask'}
color_dict = {0:(0,0,255),1:(0,255,0)}

In [14]:
def detect(frame):
    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    for (x,y,w,h) in faces:
        
        rol_frame = frame[y:y+h,x:x+w]
        resized = cv2.resize(rol_frame,(100,100)) 
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,100,100,3)) 
        reshaped = np.vstack([reshaped])
        result = classifier.predict(reshaped)

        label = np.argmax(result,axis=1)[0]
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame,label_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    return frame

In [15]:
video_capture = cv2.VideoCapture(0+cv2.CAP_DSHOW)
#converting the resolution of the output window to (640*480)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH,640)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
while True:
    
    _, frame = video_capture.read()
    canvas = detect(frame)
    cv2.imshow('result',canvas)
    key = cv2.waitKey(1)
    if key == 27:
        break
  
video_capture.release()
cv2.destroyAllWindows()